In [1]:
import pandas as pd
import numpy as np 

# STEP1. CLEAN SALES DATA (SINGLE QUARTER) #

In [2]:
# Read one file at a time into df, name df by 's'+'issue number ''
s136 = "Files/Sales/2019_2021/Issue-136-Sales-tables-March-2021-quarter.xlsx"
s136 = pd.read_excel(s136, sheet_name="Postcode", na_values='-', header=6)

In [3]:
# Rename columns
rename_cols= {'Postcode':'postcode', 
             'Dwelling Type':'dwelling_type', 
             "First Quartile Sales Price\n$'000s" : '25%_price',
             "Median Sales Price\n$'000s" : 'median_price', 
             "Third Quartile Sales Price\n'000s" : '75%_price',
             "Mean Sales Price\n$'000s" : 'mean_price',
             'Sales\nNo.':'sales_no',
             'Qtly change in Median':'Qdelta_median_price',
             'Annual change in Median':'Adelta_median_price',
             'Qtly change in Count':'Qdelta_sales_no',
             'Annual change in Count':'Adelta_sales_no'}
s136.rename(columns=rename_cols, inplace=True)

# Drop unwanted columns
s136 = s136.drop(columns=['25%_price', '75%_price'], axis=1)

s136.head(5)

,postcode,dwelling_type,median_price,mean_price,sales_no,Qdelta_median_price,Adelta_median_price,Qdelta_sales_no,Adelta_sales_no
0,2000,Total,1371.0,2794.0,184,0.1425,0.0711,0.0888,0.5862
1,2000,Strata,1371.0,2794.0,184,0.1331,0.0632,0.1018,0.6429
2,2007,Total,763.0,754.0,s,0.1713,0.0235,0.0000,0.3000
3,2007,Non Strata,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007,Strata,710.0,688.0,s,0.0906,-0.0470,-0.0769,0.2000


In [4]:
print("number of postcodes:", s136['postcode'].nunique())
print("number of rows:", s136.shape[0],'\n')
print(s136.isnull().sum())

number of postcodes: 544
number of rows: 1427 

postcode                 0
dwelling_type            0
median_price           316
mean_price             316
sales_no               316
Qdelta_median_price    316
Adelta_median_price    321
Qdelta_sales_no        316
Adelta_sales_no        321
dtype: int64


<b>IMPUTATION</b>

Roughly 1/4 of the data contains null values, which comes from:
<em>"for confidentiality, we don't report sales in any geographical area where the number of sales is 10 or less (shown as na).</em> Also: <em>"Statistics calculated from sample sizes between 10 an 30 are shown by an 's' in the relevant table".</em> This will be the primary rules of imputation:
* **For 'sales_no':**
    * Replace na with 5
    * Replace s with 20
    
* **For 'median_price' and 'mean_price'**
    * Fill na with the median of price of the same dwelling type

In [5]:
# Imputation of sales number

# na
s136.loc[s136['sales_no'].isnull(), 'sales_no'] = 5.0
s136['sales_no'].isnull().any()

# s
s136.loc[s136['sales_no'] == 's', 'sales_no'] = 20.0
s136['sales_no'] = s136['sales_no'].astype(float) # Cast type as float

print("number of na in sales_no:", s136['sales_no'].isnull().sum())
print("data type of sales_no:", s136['sales_no'].dtype)

number of na in sales_no: 0
data type of sales_no: float64


In [6]:
# Imputation of median_price (by dwelling type)

# Total
imp_total = s136.loc[(s136['median_price'].notna()) & (s136['dwelling_type']=='Total'),
                     'median_price'].median() # calculate imputer value 
    
s136.loc[(s136['median_price'].isnull()) & (s136['dwelling_type']=='Total'),
         'median_price']= imp_total #impute

# Strata 
imp_strata = s136.loc[(s136['median_price'].notna()) & (s136['dwelling_type']=='Strata'),
                     'median_price'].median() 
    
s136.loc[(s136['median_price'].isnull()) & (s136['dwelling_type']=='Strata'),
         'median_price']= imp_strata 

# Non-Strata
imp_ns = s136.loc[(s136['median_price'].notna()) & (s136['dwelling_type']=='Non Strata'),
                     'median_price'].median() 
    
s136.loc[(s136['median_price'].isnull()) & (s136['dwelling_type']=='Non Strata'),
         'median_price']= imp_ns

print("number of na in median_price:", s136['median_price'].isnull().sum())

number of na in median_price: 0


In [7]:
# Imputation of mean_price (by dwelling type)

# Total
imp_total = s136.loc[(s136['mean_price'].notna()) & (s136['dwelling_type']=='Total'),
                     'mean_price'].median() # calculate imputer value 
    
s136.loc[(s136['mean_price'].isnull()) & (s136['dwelling_type']=='Total'),
         'mean_price']= imp_total #impute

# Strata 
imp_strata = s136.loc[(s136['mean_price'].notna()) & (s136['dwelling_type']=='Strata'),
                     'mean_price'].median() 
    
s136.loc[(s136['mean_price'].isnull()) & (s136['dwelling_type']=='Strata'),
         'mean_price']= imp_strata 

# Non-Strata
imp_ns = s136.loc[(s136['mean_price'].notna()) & (s136['dwelling_type']=='Non Strata'),
                  'mean_price'].median() 
    
s136.loc[(s136['mean_price'].isnull()) & (s136['dwelling_type']=='Non Strata'),
         'mean_price']= imp_ns

print("number of na in mean_price:", s136['mean_price'].isnull().sum())

number of na in mean_price: 0


In [8]:
s136.describe().round(2)

,postcode,median_price,mean_price,sales_no,Qdelta_median_price,Adelta_median_price,Qdelta_sales_no,Adelta_sales_no
count,1427.00,1427.00,1427.00,1427.00,1111.00,1106.00,1111.00,1106.00
mean,2378.20,923.04,994.20,51.47,0.06,0.13,-0.03,0.62
std,256.21,628.89,706.76,58.94,0.14,0.20,0.41,0.85
min,2000.00,120.00,125.00,5.00,-0.90,-0.46,-0.59,-0.49
25%,2148.00,610.00,635.50,20.00,-0.01,0.03,-0.22,0.17
50%,2330.00,760.50,801.50,20.00,0.05,0.11,-0.10,0.41
75%,2575.00,960.00,1072.50,68.00,0.12,0.20,0.06,0.74
max,2880.00,6400.00,7374.00,397.00,0.78,2.04,6.50,7.00


**ADD TIME PERIOD TAG**

In [9]:
s136['key_s'] = 's136'
s136['time_period'] = '2021 Q1'
s136['year'] = '2021'
s136['quarter'] = '1'

print(s136.shape)

(1427, 13)


In [10]:
s136

,postcode,dwelling_type,median_price,mean_price,sales_no,Qdelta_median_price,Adelta_median_price,Qdelta_sales_no,Adelta_sales_no,key_s,time_period,year,quarter
0,2000,Total,1371.0,2794.0,184.0,0.1425,0.0711,0.0888,0.5862,s136,2021 Q1,2021,1
1,2000,Strata,1371.0,2794.0,184.0,0.1331,0.0632,0.1018,0.6429,s136,2021 Q1,2021,1
2,2007,Total,763.0,754.0,20.0,0.1713,0.0235,0.0000,0.3000,s136,2021 Q1,2021,1
3,2007,Non Strata,843.0,888.0,5.0,NaN,NaN,NaN,NaN,s136,2021 Q1,2021,1
4,2007,Strata,710.0,688.0,20.0,0.0906,-0.0470,-0.0769,0.2000,s136,2021 Q1,2021,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,2877,Non Strata,195.0,212.0,20.0,-0.0714,0.3636,0.5385,4.0000,s136,2021 Q1,2021,1
1423,2879,Total,760.5,801.5,5.0,NaN,NaN,NaN,NaN,s136,2021 Q1,2021,1
1424,2879,Non Strata,843.0,888.0,5.0,NaN,NaN,NaN,NaN,s136,2021 Q1,2021,1
1425,2880,Total,120.0,146.0,107.0,-0.0323,-0.0400,0.1146,0.5070,s136,2021 Q1,2021,1


### End of the sales data cleaning process ###
----

# STEP2. CLEAN RENT DATA (SINGLE QUARTER) #


In [11]:
# Read into df
r135 = "Files/Rent/Issue-135-Rent-tables-March-2021-quarter.xlsx"
r135 = pd.read_excel(r135, sheet_name="Postcode", na_values='-', header=7)

# Drop unwanted columns
r135 = r135.drop(columns=['First Quartile Weekly Rent for New Bonds\n$',
                          'Third Quartile Weekly Rent for New Bonds\n$'],
                axis=1)

# Rename columns
rename_cols= {'Postcode':'postcode',
              'Dwelling Types':'dwelling_type', 
              'Number of Bedrooms':'bed_number',
              'Median Weekly Rent for New Bonds\n$': 'median_rent_newb',
              'New Bonds Lodged\nNo.' : 'new_bonds_no',
              'Total Bonds Held\nNo.': 'total_bonds_no',
              'Quarterly change in Median Weekly Rent':'Qdelta_median_rent',
              'Annual change in Median Weekly Rent':'Adelta_median_rent',
              'Quarterly change in New Bonds Lodged':'Qdelta_new_bonds',
              'Annual change in New Bonds Lodged':'Adelta_new_bonds'}
r135.rename(columns=rename_cols,inplace=True)

r135.head(10)

FileNotFoundError: [Errno 2] No such file or directory: 'Files/Rent/Issue-135-Rent-tables-March-2021-quarter.xlsx'

In [ ]:
# Check df shape and null values
print(r135.shape)
print(r135.isnull().sum())

**NOTE:**

Note that an alarming 3/4 of the data has null values. This is because the data is broken down to very granular level - first by dwelling type (Total, house, townhouse, flat/unit, other) and then by bed_numbers (see below cell).

In [ ]:
print(r135.groupby('dwelling_type').size(),'\n')
print(r135.groupby('bed_number').size())

**AGGREGATION:**

Given the amount of na present, we'll **aggregate the data to the total level of each postcode** (i.e. only consider the total number of bonds without classifying them into dwelling types nor bedroom numbers) to avoid excessive imputation. Although we'll inevitably lose the richness of information, the main function of the rent data is to serve as an input variable in the prediction of sales, where this level of granularity is not strictly necessary.

In [ ]:
# Aggregate dwelling type and bed number, save as new df r135_ag

r135_ag = r135.loc[(r135['bed_number']=='Total') & (r135['dwelling_type']=='Total')]
r135_ag = r135_ag.drop(columns=['bed_number','dwelling_type'], axis=1) # Drop bed_number and dwelling_type

print(r135_ag.shape)
print(r135_ag.isnull().sum())

By aggregating the data, we're able to bring down the proportion of na from 3/4 to around 1/3. But there's still need for imputation. According to the data interpretation note:

<em><b>"For confidentiality, we don't report rents in any geographical area where the number of new bonds is 10 or less (shown as na). Statistics calculated from sample sizes between 10 an 30 are shown by an 's' in the relevant table"</b></em>

In [ ]:
print("number of 's' in new_bonds_no:", r135_ag.loc[r135_ag['new_bonds_no']=='s'].shape[0])
print("number of 's' in total_bonds_noA:", r135_ag.loc[r135_ag['total_bonds_no']=='s'].shape[0],"\n")
print("number of na in new_bonds_no:", r135_ag.loc[r135_ag['new_bonds_no'].isnull()].shape[0])
print("number of na in total_bonds_no:", r135_ag.loc[r135_ag['total_bonds_no'].isnull()].shape[0])

<b>IMPUTATION</b>
* For 'new_bonds_no' and 'total_bonds_no' columns:
    * Impute na with 5
    * Impute s with 20
    
* For 'median_rent_newb' column
    * Impute na with median of rents of all POAs

In [ ]:
# Impute 's' in 'new_bonds_no' and 'total_bonds_no' with 20
r135_ag.loc[r135_ag['new_bonds_no']=='s','new_bonds_no'] = 20.0
r135_ag.loc[r135_ag['total_bonds_no']=='s', 'total_bonds_no'] = 20.0

# Impute na in 'new_bonds_no' and 'total_bonds_no' with 5
r135_ag.loc[r135_ag['new_bonds_no'].isnull(),'new_bonds_no'] = 5.0
r135_ag.loc[r135_ag['total_bonds_no'].isnull(), 'total_bonds_no'] = 5.0

# Cast both variables as float (was object)
r135_ag['new_bonds_no'] = r135_ag['new_bonds_no'].astype(float)
r135_ag['total_bonds_no'] = r135_ag['total_bonds_no'].astype(float)

In [ ]:
# Impute na in 'median_rent' with median of the column
r135_ag['median_rent_newb'].fillna(r135_ag['median_rent_newb'].median(), inplace=True)

# Check na in the df again
print(r135_ag.isnull().sum())

In [ ]:
# Prepare for merging
r135_ag['key_r'] = 'r135' # Add key

# Set postcode as index
r135_ag = r135_ag.set_index('postcode')
r135_ag.head(1)

### End of the rent data cleaning process ##
----



# STEP3. JOIN SALES AND RENT DATA OF THE SAME QUARTER #

### !! Please note !! ###
**A. the issue number of the sales and rent table of the same quarter are different**
<br>For example, for Q1 2021, the issue number of the sales table is 136 and that of the rent table is 135. The resulting data frames are s136 and r135_ag. For the same quarter, the issue number of sales table is always that of the rent+1 </br>

**B. the sales table and the rent table contains different numbers of postcodes**
<br>For example, s136 (sales table) contains 544 unique postcodes while r135 (rent table) contains 617. We will merge the rent table into the sales table, i.e. only keep postcodes that are available in the sales table </br>

In [18]:
# Merge the rent df into the sales df of the same quarter (Q1 2021)
q121 = s136.join(r135_ag, on='postcode')
q121.head(1)

# Note: naming convention of merged quarterly sales+rent df
# quarter in lowercase + 2digit year
# e.g. Q4 2018 will be q418

,postcode,dwelling_type,median_price,mean_price,sales_no,Qdelta_median_price,Adelta_median_price,Qdelta_sales_no,Adelta_sales_no,key_s,...,year,quarter,median_rent_newb,new_bonds_no,total_bonds_no,Qdelta_median_rent,Adelta_median_rent,Qdelta_new_bonds,Adelta_new_bonds,key_r
0,2000,Total,1371.0,2794.0,184.0,0.1425,0.0711,0.0888,0.5862,s136,...,2021,1,600.0,1469.0,9327.0,0.0909,-0.1429,-0.1384,0.1943,r135


In [19]:
q121['postcode'].nunique()

544

# STEP4 Repeat above process to the sales and rent data of the rest quarters #

**@Chris** - I'm assuming that we will be able to do this easily with the functions created from step 1-3?

# STEP5 Concact merged sales and rent quarterly df into one master df #

In [ ]:
# NOTE: Dummy code here 
frames = [q118, q218, q318, q418, 
          q119, q219, q319, q419,
          q120, q220, q320, q420,
          q121]
sr_master = pd.concat(frames)

# STEP 6 Add postcode to LGA Mapping to the master df #

In [22]:
# Read the lga-postcode data into df
mapping = "Files/Area/Postcode_and_LGA.xlsx"
lga_poa = pd.read_excel(mapping, sheet_name="SuburbLGA", 
                        usecols=['lganame','councilnam','suburbname','postcode'])

# Rename columns 
rename_cols= {'lganame':'LGA',
              'councilnam':'council', 
              'suburbname':'suburb'}
lga_poa.rename(columns=rename_cols,inplace=True)

lga_poa.head()

,LGA,council,suburb,postcode
0,ALBURY CITY,ALBURY CITY COUNCIL,ALBURY,2640.0
1,ALBURY CITY,ALBURY CITY COUNCIL,EAST ALBURY,2640.0
2,ALBURY CITY,ALBURY CITY COUNCIL,ETTAMOGAH,2640.0
3,ALBURY CITY,ALBURY CITY COUNCIL,GLENROY,2640.0
4,ALBURY CITY,ALBURY CITY COUNCIL,HAMILTON VALLEY,2641.0


In [24]:
# Check the number of postcodes and null values

print(lga_poa['postcode'].nunique())
print(lga_poa.isnull().sum())

622
LGA         12
council     12
suburb       0
postcode    12
dtype: int64


In [31]:
# Check null values in the postcode
lga_poa.loc[lga_poa['postcode'].isnull()]

,LGA,council,suburb,postcode
204,BAYSIDE,BAYSIDE COUNCIL,BOTANY BAY,NaN
462,BLUE MOUNTAINS,BLUE MOUNTAINS CITY COUNCIL,BLUE MOUNTAINS NATIONAL PARK,NaN
2370,LAKE MACQUARIE,LAKE MACQUARIE CITY COUNCIL,LAKE MACQUARIE,NaN
2860,MID-COAST,MID-COAST COUNCIL,PORT STEPHENS,NaN
3340,NORTHERN BEACHES,NORTHERN BEACHES COUNCIL,KU-RING-GAI CHASE,NaN
3345,NORTHERN BEACHES,NORTHERN BEACHES COUNCIL,MIDDLE HARBOUR,NaN
3357,NORTHERN BEACHES,NORTHERN BEACHES COUNCIL,PITTWATER,NaN
3598,PORT STEPHENS,PORT STEPHENS COUNCIL,PORT STEPHENS,NaN
4578,UNINCORPORATED,UNINCORPORATED,BOTANY BAY,NaN
4593,UNINCORPORATED,UNINCORPORATED,MIDDLE HARBOUR,NaN


In [30]:
# unincorporated LGA?
lga_poa.loc[lga_poa['LGA']=='UNINCORPORATED']

,LGA,council,suburb,postcode
4574,UNINCORPORATED,UNINCORPORATED,ABBOTSFORD,2046.0
4575,UNINCORPORATED,UNINCORPORATED,BALMAIN,2041.0
4576,UNINCORPORATED,UNINCORPORATED,BARANGAROO,2000.0
4577,UNINCORPORATED,UNINCORPORATED,BIRCHGROVE,2041.0
4578,UNINCORPORATED,UNINCORPORATED,BOTANY BAY,NaN
4579,UNINCORPORATED,UNINCORPORATED FAR WEST,BROKEN HILL,2880.0
4580,UNINCORPORATED,UNINCORPORATED FAR WEST,BROUGHAMS GATE,2880.0
4581,UNINCORPORATED,UNINCORPORATED,DAWES POINT,2000.0
4582,UNINCORPORATED,UNINCORPORATED,DOUBLE BAY,2028.0
4583,UNINCORPORATED,UNINCORPORATED,DRUMMOYNE,2047.0
